<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="160" />

<br/>

# Quick Start for Contextual AI Platform using the Python Client

Contextual AI lets you create and use generative AI agents. This notebook gives a quick start for creating a Retrieval-Augmented Generation (RAG) agent for a financial use case. The agent will answer questions based on the documents provided, but avoid any forward looking statements, e.g., Tell me about sales in 2028. This notebook uses the python client.

This notebook covers the following steps:
- Creating a Datastore
- Ingesting Documents
- Creating a RAG Agent
- Querying a RAG Agent
- Providing feedback on a query response
- Collecting metrics on a RAG agent

The notebook can be run in under 15 minutes. The full documentation is available at [docs.contextual.ai](https://docs.contextual.ai/)

To run this notebook interactively, you can open it in Google Colab. However, make sure to copy over the additional files.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ContextualAI/examples/blob/main/01-getting-started/quick-start.ipynb)

In [ ]:
!pip install contextual-client

In [2]:
import os
import requests
from pathlib import Path
from typing import List, Optional, Dict
import pandas as pd
from contextual import ContextualAI

In [3]:
#os.environ["CONTEXTUAL_API_KEY"] = API_KEY  # You can store the API key is stored as the environment variable 

client = ContextualAI(
    api_key=os.environ.get("CONTEXTUAL_API_KEY"),  # This is the default and can be omitted
)

REQUEST_URL = 'https://api.contextual.ai/v1'


## Step 1: Create your Datastore


You will need to first create a datastore for your agent using the  /datastores endpoint. A datastore is secure storage for data. Each agent will have it's own datastore for storing data securely.

In [ ]:
result = client.datastores.create(name="Demo_fin_rag")
datastore_id = result.id
print(f"Datastore ID: {datastore_id}")

## Step 2: Ingest Documents into your Datastore

You can now ingest documents into your Agent's datastore using the /datastores endpoint. Documents must be in PDF or HTML format.


I am using an example PDF. You can also use your own documents here. If you have very large documents (hundreds of pages), processing can take longer.

In [2]:
if not os.path.exists('data'):
    os.makedirs('data')

if not os.path.exists('data/Apple.pdf'):
    print(f"Fetching data/Apple.pdf")
    response = requests.get("https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/01-getting-started/data/Apple.pdf")
    with open('data/Apple.pdf', 'wb') as f:
        f.write(response.content)

Let's ingest the file

In [ ]:
with open('data/Apple.pdf', 'rb') as f:
    ingestion_result = client.datastores.documents.ingest(datastore_id, file=f)
    document_id = ingestion_result.id
    print(f"Successfully uploaded to datastore {datastore_id}")

Once ingested, you can view the list of documents, see their metadata, and also delete documents.

In [ ]:
metadata = client.datastores.documents.metadata(datastore_id = datastore_id, document_id = document_id)
print("Document metadata:", metadata)

## Step 3: Create your Agent

Next let's create the Agent and modify it to our needs.


Some additional parameters include setting a system prompt or using a previously tuned model.

`system_prompt` is used for the instructions that your RAG system references when generating responses. Note that we do not guarantee that the system will follow these instructions exactly.

`llm_model_id` is the optional model ID of a tuned model to use for generation. Contextual AI will use a default model if none is specified.

In [7]:
system_prompt = '''
You are an AI assistant specialized in financial analysis and reporting. Your responses should be precise, accurate, and sourced exclusively from official financial documentation provided to you. Please follow these guidelines:

Data Analysis & Response Quality:
* Only use information explicitly stated in provided documentation (e.g., earnings releases, financial statements, investor presentations)
* Present comparative analyses using structured formats with tables and bullet points where appropriate
* Include specific period-over-period comparisons (quarter-over-quarter, year-over-year) when relevant
* Maintain consistency in numerical presentations (e.g., consistent units, decimal places)
* Flag any one-time items or special charges that impact comparability

Technical Accuracy:
* Use industry-standard financial terminology
* Define specialized acronyms on first use
* Never interchange distinct financial terms (e.g., revenue, profit, income, cash flow)
* Always include units with numerical values
* Pay attention to fiscal vs. calendar year distinctions
* Present monetary values with appropriate scale (millions/billions)

Response Format:
* Begin with a high-level summary of key findings when analyzing data
* Structure detailed analyses in clear, hierarchical formats
* Use markdown for lists, tables, and emphasized text
* Maintain a professional, analytical tone
* Present quantitative data in consistent formats (e.g., basis points for ratios)

Critical Guidelines:
* Avoid opinions, speculation, or assumptions
* If information is unavailable or irrelevant, clearly state this without additional commentary
* Answer questions directly, then stop
* Do not reference source document names or file types in responses
* Focus only on information that directly answers the query

For any analysis, provide comprehensive insights using all relevant available information while maintaining strict adherence to these guidelines and focusing on delivering clear, actionable information.
'''


Let's create our agent. 

In [ ]:
app_response = client.agents.create(
    name="Demo-Finance Forward Looking",
    description="Research Agent using only Historical Information",
    system_prompt=system_prompt,
    datastore_ids=[datastore_id]
)
agent_id= app_response.id
print(f"Agent ID created: {agent_id}")

In [6]:
## (Optional) Step 3a: Connect to a different agent
# agent_id = ''

## Step 4: Query your Agent

Let's query our agent to see if its working. The required information is the agent_id and messages.  

Optional information includes parameters for streaming, conversation_id, and model_id if using a different fine tuned model.

**Note:** It may take a few minutes for the document to be ingested and processed. The Assistant will give a detailed answer once the documents are ingested.

In [ ]:
query_result = client.agents.query.create(
    agent_id=agent_id,
    messages=[{
        "content": "what was the sales for Apple in 2022",
        "role": "user"
    }]
)
print(query_result.message.content)

## Step 5: Provide feedback on queries

Once our agent responds back with a response to a query, we can provide feedback on the answer. The required information is the agent_id, message_id and feedback in the form of "thumbs_up", "thumbs_down" or "flagged", and can set this to "removed" to undo previous feedback.  

Optionally, we can include a free-form explanation string. Lastly, we can choose to provide feedback on a specific retrieval by specifying a content_id.

In [ ]:
feedback_result = client.agents.query.feedback(
    agent_id=agent_id,
    message_id=query_result.message_id,
    feedback="thumbs_up",
    explanation="Great answer"
    # content_id=query_result.retrieval_contents[0].content_id
)

print("Feedback submitted successfully")

## Step 6: Collect metrics on an agent

Once our agent is deployed to end users, we can query for data on usage and user-provided feedback, so we can make data-driven improvements and optimizations. The required information is the agent_id.

Optionally, we can filter messages using `created_after` or `created_before` parameters, specify a `limit` (up to a 1000, defaults to 100) or specify an `offset` for pagination.

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML, display

# Get metrics from the API
metrics = client.agents.query.metrics(agent_id=agent_id)
messages_list = metrics.messages

# Create DataFrame from messages list
df = pd.DataFrame(messages_list)

# Calculate metrics
total_messages = len(df)
feedback_counts = df['feedback'].value_counts().to_dict()
feedback_counts['no_feedback'] = df['feedback'].isna().sum()
flagged_count = df['issues'].apply(lambda x: x == '{}').sum()  # Count non-empty issues

# Display metrics
print("=== Metrics Dashboard ===")
print(f"Total Messages: {total_messages}")
print("\nFeedback Distribution:")
for feedback, count in feedback_counts.items():
    print(f"- {feedback}: {count}")
print(f"\nFlagged Messages: {flagged_count}")

# Create a bar plot of feedback distribution
plt.figure(figsize=(10, 6))
pd.Series(feedback_counts).plot(kind='bar')
plt.title('Feedback Distribution')
plt.xlabel('Feedback Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Create display DataFrame with relevant columns
display_df = df[['question', 'feedback', 'created_at', 'detail']].copy()
display_df['created_at'] = pd.to_datetime(display_df['created_at'])

# Format the display DataFrame
display(HTML("<h3>Message Details:</h3>"))
pd.set_option('display.max_colwidth', None)  # Show full content of cells
display(display_df)

# Function to export to CSV
def export_to_csv(filename='metrics_export.csv'):
    df.to_csv(filename, index=False)
    print(f"Data exported to {filename}")

# To export:
# export_to_csv()

## Next Steps

In this Notebook, you have started with creating a RAG agent in the finance domain. Check out our other notebooks for a deeper dive into the Contextual AI platform, including how you evaluate and tune your agent. Finally, reach out to your account team if you have further questions or issues.